#Synthesis

In [2]:
import sys
import os
import cv2
import tensorflow.keras.backend as K
import numpy as np
from numpy import asarray
from core.networks_simple import UNET
from PIL import Image as im
K.set_image_data_format('channels_first')
os.environ['KERAS_BACKEND'] = "tensorflow"

def Synthesis(width, height,path1, path2):
  res_arr = []
  vid_arr = np.zeros(shape=(2, height, width, 3), dtype="uint8")
  # image1 = im.open(path1)
  image1 = cv2.cvtColor(cv2.imread(path1), cv2.COLOR_BGR2RGB)
  # image2 = im.open(path2)
  image2 = cv2.cvtColor(cv2.imread(path2), cv2.COLOR_BGR2RGB)
  vid_arr[0] = cv2.resize(asarray(image1), (width, height))
  vid_arr[1] = cv2.resize(asarray(image2), (width, height))
  model = UNET((6, height, width))
  model.load_weights('./model_weights_for 512x256 5ep simple unet/model_weights.hdf5')
  pred = model.predict(np.expand_dims(np.transpose(np.concatenate((vid_arr[0], vid_arr[1]), axis=2)/255., (2,0,1)), axis=0))
  res = (np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")
  # img = im.fromarray(res)
  # img.save(out_path)
  res_arr.append(res)
  return res_arr


In [3]:
def Synthesis_rec(path1, path2, n, last):
  image1 = cv2.cvtColor(cv2.imread(path1), cv2.COLOR_BGR2RGB)
  image2 = cv2.cvtColor(cv2.imread(path2), cv2.COLOR_BGR2RGB)
  height, width, _ = image1.shape
  vid_arr = np.zeros(shape=(2, height, width, 3), dtype="uint8")
  vid_arr[0] = cv2.resize(asarray(image1), (width, height))
  vid_arr[1] = cv2.resize(asarray(image2), (width, height))
  model = UNET((6, height, width))
  model.load_weights('./model_weights_for 512x256 5ep simple unet/model_weights.hdf5')
  def recursive_calls(frame1, frame2, num_recursions):
    if num_recursions == 0:
      yield frame1
    else:
      pred = model.predict(np.expand_dims(np.transpose(np.concatenate((frame1, frame2), axis=2)/255., (2,0,1)), axis=0))
      mid_frame = (np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")
      yield from recursive_calls(frame1, mid_frame, num_recursions - 1)
      yield from recursive_calls(mid_frame, frame2, num_recursions - 1)
  result = list(recursive_calls(vid_arr[0], vid_arr[1], n))
  if last:
    result.append(vid_arr[1])
  return result
  # for i,img in enumerate(result):
  #   image = im.fromarray(img)
  #   image.save(f'./{out_path}/{i}.png')

#UTILS

In [5]:
from PIL import Image
import glob
def helper(path):
  for filename in glob.glob(f'{path}/*.png'):
    img = Image.open(filename)
    w, h = img.size
    print(f"Original Size: Width-{w} Height-{h}")
    w = int(w / 32) * 32
    h = int(h / 32) * 32
    print(f"Modified Size: Width-{w} Height-{h}")
    resized_img = img.resize((w, h), Image.ANTIALIAS)
    resized_img.save(filename)

In [6]:
path = './Drive/realtime/'
helper(path)

Original Size: Width-352 Height-640
Modified Size: Width-352 Height-640
Original Size: Width-352 Height-640
Modified Size: Width-352 Height-640


#Metrics

In [7]:
import math
from skimage.metrics import structural_similarity as compare_ssim
def calculateMetrics(gt, result):
  psnr(gt, result)
  ssim = calculate_ssim(gt, result)
  print(f"SSIM: {ssim}%")

def psnr(img1, img2):
  mse = np.mean((img1 - img2) ** 2)
  if mse == 0:
    return 100
  # print(f"MSE: {mse}")
  PIXEL_MAX = 255.0
  psnr_val = 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
  print(f"PSNR: {psnr_val}")
  return psnr_val

def calculate_ssim(gt, result):
  grayGT = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)
  grayRes = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
  (score, diff) = compare_ssim(grayGT, grayRes, full=True)
  diff = (diff * 255).astype("uint8")
  return score

#Color Correction

In [8]:
from PIL import Image
def adjuster(r1, r2, diff, gt, image):
  mask1 = np.where((diff >= r1) & (diff < (r1 + 1)))
  mask2 = np.where((diff >= r2) & (diff < (r2 + 1)))
  print(mask1[0].size, mask2[0].size)
  image[mask1] += r1
  image[mask2] -= r1
  diff_array = np.absolute(np.array(gt) - np.array(image))
  unique, counts = np.unique(diff_array, return_counts=True)
  print(unique)
  print(counts)
  print(f"For Val: {r1} and {r2 - 256}")
  calculateMetrics(gt, image)
  modified_img = Image.fromarray(image, 'RGB')
  return modified_img

Dynamic Interpolation

In [9]:
path1 = './Drive/realtime/out-001.png'
path2 = './Drive/realtime/out-003.png'
img1 = cv2.imread(path1)
img2 = cv2.imread(path2)
psnr_val = psnr(img1, img2)

PSNR: 35.717255085878634


In [10]:
def dynamic_rec(path1, path2, last):
  img1 = cv2.imread(path1)
  img2 = cv2.imread(path2)
  psnr_val = psnr(img1, img2)
  height, width, _ = img1.shape
  print(psnr_val)
  result = []
  if(psnr_val > 38):
    res = Synthesis_rec(path1, path2, 1, last) #Synthesis(width, height, path1, path2)
  elif(psnr_val > 35 and psnr_val <= 38):
    res = Synthesis_rec(path1, path2, 2, last)
  elif(psnr_val > 32 and psnr_val <= 35):
    res = Synthesis_rec(path1, path2, 3, last)
  elif(psnr_val > 30 and psnr_val <= 32):
    res = Synthesis_rec(path1, path2, 4, last)
  elif(psnr_val > 27 and psnr_val <= 30):
    res = Synthesis_rec(path1, path2, 5, last)
  elif(psnr_val <= 27):
    res = Synthesis_rec(path1, path2, 6, last)
  for i in res:
    result.append(i)
  return result

#Runner

In [13]:
import os
directory = './Drive/realtime'
out_path = './Drive/realtime'
frameNames = []
for file in os.listdir(directory):
  frameNames.append(file)
# print(frameNames)
result = []
last = False
for i in range(len(frameNames)-1):
  print(directory+'/'+frameNames[i], directory+'/'+frameNames[i+1])
  if(i == len(frameNames)-2):
    last = True
  res = dynamic_rec(directory+'/'+frameNames[i], directory+'/'+frameNames[i+1], last)
  for i in res:
    result.append(i)
for i,img in enumerate(result):
  image = im.fromarray(img)
  image.save(f'./{out_path}/{i}.png')


./Drive/realtime/out-001.png ./Drive/realtime/out-003.png
PSNR: 35.717255085878634
35.717255085878634
